# Sample Size Estimation

There are actually two different cases we need to consider:

1. Sample estimation for binomial variables (proportions).
2. Sample estimation for continuous variables (means).

We can get general formulas for both sample size estimation cases from `(1)` [here](https://www.itl.nist.gov/div898/handbook/prc/section2/prc242.htm) and `(2)` [here](https://www.itl.nist.gov/div898/handbook/prc/section2/prc222.htm).

For proportions, this looks like:

$$ n \geq \left(\frac{Z_{1-\alpha}\sqrt{p_{1}(1 - p{1})} + Z_{1-\beta}\sqrt{p_{2}(1 - p{2})}}{\mu^*}\right)^2 $$

For means, this looks like:

$$ n \geq \left(\cfrac{Z_{\alpha} + Z_{1-\beta}}{\cfrac{\mu^*}{\sigma}}\right)^2 $$

The quantities are as follows:

- $ Z_{1-\alpha} $: the Z-score corresponding to our significance level $ 1-\alpha $.
- $ Z_{1-\beta} $: the Z-score corresponding to $ 1-\beta $ where $ 1-\beta $ is our statistical power.
- $ \mu^* $: the minimum detectable effect (MDE)
- $ p_{1} $: the expected click through rate of the control
- $ p_{2} $: the expected click through rate of the test
- $ \sigma $: the (pooled) standard deviation of the samples.

Note: The above equations above are for a one-tailed tests, which should be our primary mode of testing for A/B tests. For two-tailed tests, you should estimate by using the Z-score for $ 1-\frac{\alpha}{2} $.

Okay, so now we have the theory at hand, let's code up a simple sample size estimation calculator we can use!

In [99]:
import scipy.stats as st
import math

def get_sample_estimate(sig, power, mde, p1, sigma1=None, var_type='binomial', tail=1):
    """
    sig: stat sig level (usually 0.05 or 0.01)
    power: stat power required
    mde: minimum detectable effect
    p1: value of control group test statistic
    sigma1: value of control group test statistic std (required only for continuous var_type)
    var_type: one of "binomial" or "continuous"
    tail: 1 or 2
    """
    
    # Calculate the Z_alpha given the sig and tail
    if tail == 1:
        z_alpha = st.norm.ppf(1 - sig)
    elif tail == 2:
        z_alpha = st.norm.ppf(1 - sig/2)
    else:
        raise Exception("tail can only be 1 or 2")
        
    z_beta = st.norm.ppf(power)
    p2 = p1 + mde
    
    # Now we are ready to estimate the number of samples    
    if var_type == 'binomial':
        n = ((z_alpha*math.sqrt(p1*(1-p1)) + z_beta*math.sqrt(p2*(1-p2))) / mde)**2
    elif var_type == 'continuous':
        n = ((z_alpha + z_beta) / (mde/sigma1))**2
    else:
        raise Exception("var_type can only be binomial or continuous")
        
    return n

# Running the Experiment

In [100]:
# Calculate sample size required:
p1 = 0.04
mde = 0.01
sig = 0.01
power = 0.8

n = get_sample_estimate(sig, power, mde, p1, tail=1)
print("{} samples required".format(n))

4086.9977932231523 samples required


In [101]:
import random

def draw_samples(p, n):
    """
    Draw n samples with probability p of being 1 (and 1-p of being 0)
    """
    
    def draw_sample(p):
        return 1 if random.uniform(0, 1.0) <= p else 0
    
    o = []
    for i in range(0, n):
        o.append(draw_sample(p))
    
    return o

In [139]:
control_group = draw_samples(0.04, 5000)

# To simulate the landing page having a 1.5% lift, we draw samples with p = 0.055 for our test group
test_group = draw_samples(0.055, 5000)

# Evaluating Results

In [140]:
# Calculating click through rate for test and control group
ctr_ctrl = sum(control_group) / len(control_group)
ctr_test = sum(test_group) / len(test_group)

print("Control CTR: {}%".format(ctr_ctrl * 100))
print("Test CTR: {}%".format(ctr_test * 100))

Control CTR: 4.24%
Test CTR: 5.779999999999999%


In [144]:
f_obs = [len(list(filter(lambda v: v == 0, test_group))), len(list(filter(lambda v: v == 1, test_group)))]
f_exp = [len(list(filter(lambda v: v == 0, control_group))), len(list(filter(lambda v: v == 1, control_group)))]

st.chisquare(f_obs, f_exp)

Power_divergenceResult(statistic=29.205285225642722, pvalue=6.510138201463369e-08)

As a matter of principle, we should also check the statistical power we obtain for this test, although it should not have am impact on our results.

There are two ways to do this: first, we could run simulations in order to see in what percentage of simulated experiments did our test reject the null hypothesis (given we know that the new landing page increases the CTR), or, there is also an approximation formula we can use (taken from [here](https://math.stackexchange.com/a/3262418)).

$$ \lambda = n \sum_{i=0}^{1}\frac{(O_{i} - E_{i})^2}{E_{i}} $$

In [168]:
r_int = []
for i in (0, 1):
    
    o = len(list(filter(lambda v: v == i, test_group)))
    e = len(list(filter(lambda v: v == i, control_group)))
    
    r_int.append((o - e)**2 / e)
    
print("Power: ", st.chi2.cdf(sum(r_int) * 5000, 1))

Power:  1.0
